# Condition number

Let $T \in \mathbb{R}^{I_1 \times \ldots \times I_L}$ be a tensor and suppose we want to find a rank-$R$ approximation $\tilde{T} = (W^{(1)}, \ldots, W^{(L)}) \cdot I$ for $T$. Usually any CPD solver (Tensor Fox included) tries to find an approximation that minimizes the error function

$$F(W^{(1)}, \ldots, W^{(L)}) = \frac{1}{2} \| T - \tilde{T} \|^2.$$

Note that $\tilde{T}$ lies in the input space, not the output space. Therefore we are actually minimizing the backward error of the problem. In order to have a small forward error we also need that the solution is well conditioned. In this case the forward error is defined as

$$\| (T_1, \ldots, T_R) - (\tilde{T}_1, \ldots, \tilde{T}_R) \| = \min_{s \in \mathfrak{S}_R} \sqrt{ \|T_1 - \tilde{T}_{s(1)} \|^2 + \ldots + \| T_R - \tilde{T}_{s(R)}) \|^2 },$$
where $\mathfrak{S}_R$ is the group of $R$ permutations, $T = \sum_{r=1}^R$ and $\tilde{T} = \sum_{r=1}^R \tilde{T}_r$. In other words, the forward error is the accumulated error of the rank one terms. Furthermore, it is necessary to minimize over all possible permutations because the CPD computed may have the rank one terms permuted in a different order of the original tensor.

The condition number associated to the problem of the CPD computation is then

$$\kappa(T_1, \ldots, T_R) = \lim_{\epsilon \to 0} \max_{\tilde{T} \in D(T, \epsilon, R)} \frac{ \| (T_1, \ldots, T_R) - (\tilde{T}_1, \ldots, \tilde{T}_R) \| }{ \| T - \tilde{T} \| }, $$
where $D(T, \epsilon, R) \subset \mathbb{R}^{I_1 \times \ldots \times I_L}$ is the intersection between the $\epsilon$-ball centered at $T$ and the set of rank-$R$ tensors. 

For more about the theory described here I recommend reading the following articles:

[1] P. Breiding and . Vannieuwenhoven, *A Riemannian Trust Region Method for the Canonical Tensor Rank Approximation Problem*, SIAM J. Optim., 28(3), 2435-2465.

[2] P. Breiding and N. Vannieuwenhoven, *The Condition Number of Join Decompositions*, arXiv:1611.08117v3 (2018).

In [1]:
import numpy as np
import TensorFox as tfx

In [2]:
# Create a random rank-R tensor. Tensor Fox has a function for this purpose.
R = 5
dims = [20, 30, 40] 
T, orig_factors = tfx.aux.gen_rand_tensor(dims, R)

In [3]:
# First let's see what is the condition number of the original tensor.
cond = tfx.mlinalg.cond(orig_factors)
print('cond(T) =', cond)

cond(T) = 2.052731923992039


Random tensors generated this way (i.e., their factor matrices have entries draw from the standard Gaussian distribution) usually have low condition numbers. Remember that condition numbers are useful to bound the forward error with the classic rule of thumb in numerical analysis:

$$\underbrace{\| (T_1, \ldots, T_R) - (\tilde{T}_1, \ldots, \tilde{T}_R) \|}_{\text{forward error}} \leq \underbrace{\kappa(T_1, \ldots, T_R)}_{\text{condition number}} \cdot \underbrace{\| T - \tilde{T} \|}_{\text{backward error}}.$$

Now we compute an approximated CPD for $T$.

In [4]:
# Compute CPD.
factors, T_approx, output = tfx.cpd(T, R)

In [5]:
# Display backward error and condition number of the approximation.
backward_error = output.rel_error * np.linalg.norm(T)
condition_number = tfx.mlinalg.cond(factors)
print('backward error =', backward_error)
print('cond(T_approx)=', condition_number)

backward error = 0.00047030291857279007
cond(T_approx)= 2.052731861398798


We can see that the backward error is of order $\mathcal{O}(10^{-4})$ and that the condition number is very close to the original one, which indicates the solution is indeed good. With these two values we can bound the forward error.

In [6]:
print('forward error <=', condition_number, '*', backward_error, '=', condition_number*backward_error)

forward error <= 2.052731861398798 * 0.00047030291857279007 = 0.0009654057854632108


Finally, we compute the actual forward error of this solution. We remark that the following function generates $1000$ random permutations and keep the best one, so the forward error showed is just an estimate. To get better estimates you can set the keyword *trials* to a higher value. However be warned that this function is costly.

In [7]:
# new_factors are the factors after permutation, and idx is the permutation index.
forward_error, new_factors, idx = tfx.mlinalg.forward_error(orig_factors, factors, trials=10000)
print('forward error =', forward_error)

forward error = 0.0004702711368125619
